In [1]:
from tagger.core.mongo.models.order import Order, OrderCodeMap
from tagger.models import ExtendedOrder


import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
valid_orders = Order.objects(orderstatus__nin=[
    "CREATED",
    "RECREATED",
    "PAYMENT_PENDING"
]).all()

In [6]:
for o in valid_orders:
    _code = o.code
    code = _code.replace("ORD-", "") if _code[:4] == "ORD-" else None

    if ExtendedOrder.objects.filter(order_id=o.id).count() > 0:
        eo = ExtendedOrder.objects.get(order_id=o.id)
        if eo.code != code and code is not None:
            print(f"Changing extended order {o.id} code from {eo.code} to {code}")
            eo.code = code
            eo.save()
        elif eo.code is None:
            new_code = ExtendedOrder.generate_usable_code()
            print(f"Changing extended order {o.id} code from {eo.code} to {new_code}")
            eo.code = new_code
            eo.save()
    else:
        e = ExtendedOrder.make_from(o, code=code)
        print(f"Order {o.id} has been extended with code {e.code}! (Given code: {code})")

Changing extended order 6189236026685ff051755ba8 code from TMWR7 to UA3TM
Order 61892b5e26685ff051755bca has been extended with code NKTA3! (Given code: NKTA3)
Order 61893d7a26685ff051755c0f has been extended with code BTMJY! (Given code: BTMJY)
Order 6189414326685ff051755c18 has been extended with code KTDY3! (Given code: KTDY3)
Order 61899c9f26685ff051755c53 has been extended with code TAQXD! (Given code: TAQXD)
Order 6189baa126685ff051755c70 has been extended with code L6MTY! (Given code: L6MTY)
Order 6189f73b26685ff051755cb4 has been extended with code 9LCLN! (Given code: 9LCLN)
Order 6189f88826685ff051755cb7 has been extended with code 6AEGG! (Given code: 6AEGG)
Order 618a1a8126685ff051755ce3 has been extended with code 7KXXG! (Given code: 7KXXG)
Order 618a1c7626685ff051755cec has been extended with code RVRG3! (Given code: RVRG3)


/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-08 13:51:26.430000) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-08 15:08:42.520000) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-08

Order 618a231f26685ff051755cf9 has been extended with code J4A4N! (Given code: J4A4N)
Order 618a33a726685ff051755d08 has been extended with code CAU93! (Given code: CAU93)
Order 618a399726685ff051755d16 has been extended with code EHQB6! (Given code: EHQB6)
Order 618a3c6e26685ff051755d2f has been extended with code CGTCW! (Given code: CGTCW)
Order 618a3df226685ff051755d37 has been extended with code HUFLX! (Given code: HUFLX)
Order 618a3e6826685ff051755d3c has been extended with code UHYVG! (Given code: UHYVG)
Order 618a40c526685ff051755d44 has been extended with code Q9QVQ! (Given code: Q9QVQ)
Order 618a452226685ff051755d4a has been extended with code YHYFJ! (Given code: YHYFJ)
Order 618a4c2f26685ff051755d56 has been extended with code JDB6V! (Given code: JDB6V)
Order 618a4ca126685ff051755d58 has been extended with code 9YUM9! (Given code: 9YUM9)


/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-09 08:39:03.733000) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-09 09:04:23.947000) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-09

Order 618a5c2826685ff051755d81 has been extended with code KW43Y! (Given code: KW43Y)
Order 618a643826685ff051755d9e has been extended with code 48MEJ! (Given code: 48MEJ)
Order 618a64b026685ff051755da2 has been extended with code GJGRK! (Given code: GJGRK)
Order 618a667d26685ff051755dad has been extended with code XWLTM! (Given code: XWLTM)
Order 618a691b26685ff051755db6 has been extended with code 6RBWW! (Given code: 6RBWW)
Order 618a827426685ff051755e00 has been extended with code M34MY! (Given code: M34MY)
Order 618a91e826685ff051755e26 has been extended with code PPUHY! (Given code: PPUHY)
Order 618a933c26685ff051755e2c has been extended with code B49G4! (Given code: B49G4)
Order 618ab21e26685ff051755e4a has been extended with code 47EDC! (Given code: 47EDC)
Order 618b242d26685ff051755e9c has been extended with code 8G7L9! (Given code: 8G7L9)
Order 618b318b26685ff051755ea9 has been extended with code RCTHP! (Given code: RCTHP)


...

/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-13 12:58:12.046000) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-13 13:16:12.629000) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-13

Order 618fb66426685ff0517566c9 has been extended with code 4LATL! (Given code: 4LATL)
Order 618fba9c26685ff0517566d8 has been extended with code 7ULLV! (Given code: 7ULLV)
Order 618fbc4226685ff0517566e7 has been extended with code 33VUR! (Given code: 33VUR)
Order 618fbd8026685ff0517566f3 has been extended with code QC6LJ! (Given code: QC6LJ)
Order 618fc02e26685ff0517566fc has been extended with code PTEE3! (Given code: PTEE3)
Order 618fc68826685ff051756721 has been extended with code X3KAU! (Given code: X3KAU)
Order 618fd71a26685ff051756767 has been extended with code MHJAP! (Given code: MHJAP)
Order 61904d3626685ff0517567a5 has been extended with code LDVMJ! (Given code: LDVMJ)


/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-14 00:42:32.535000) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-14 00:47:25.930000) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-14

Order 61905b7826685ff0517567b8 has been extended with code T4CKJ! (Given code: T4CKJ)
Order 61905c9d26685ff0517567bc has been extended with code AQRE4! (Given code: AQRE4)
Order 61906e9626685ff0517567cc has been extended with code VPQUA! (Given code: VPQUA)
Order 61907c9a26685ff0517567da has been extended with code DQEY6! (Given code: DQEY6)
Order 61908c1226685ff0517567e1 has been extended with code XFNV4! (Given code: XFNV4)
Order 61908e6426685ff0517567e6 has been extended with code 7XCTA! (Given code: 7XCTA)
Order 6190938e26685ff0517567ea has been extended with code 64XYT! (Given code: 64XYT)
Order 6190a16026685ff0517567fb has been extended with code WN73Y! (Given code: WN73Y)
Order 6190a6d326685ff051756800 has been extended with code UXCHH! (Given code: UXCHH)
Order 6190c09326685ff051756810 has been extended with code FMF3M! (Given code: FMF3M)
Order 6190d47326685ff051756819 has been extended with code VYQQ3! (Given code: VYQQ3)
Order 6190f92526685ff05175682a has been extended with 

/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-14 09:18:43.811000) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-14 11:55:17.651000) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField ExtendedOrder.ordered_at received a naive datetime (2021-11-14

In [7]:
ExtendedOrder.objects.filter(code=None)

<QuerySet []>